# GETTIN
## Planejamento de Equipamentos Escolares
## Método: k-Median
### Versão: 0.5
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


#### Modeling and Creating Decision Variables

\begin{split}& \mbox{minimize} \quad    & \sum_{i=1}^n \sum_{j=1}^m c_{ij} x_{ij} &     \\
& \mbox{subject to:} \quad & \sum_{j=1}^m x_{ij} =1 &  \mbox{ for } i=1,\cdots,n\\
&   & \sum_{j=1}^m y_{j} = k                        &   \\
&   & x_{ij} \leq y_j                               & \mbox{ for }  i=1,\cdots,n; j=1,\cdots,m  \\
&   & x_{ij} \in \{ 0,1 \}                          & \mbox{ for }  i=1,\cdots,n; j=1,\cdots,m \\
&   & y_j \in \{ 0,1 \}                             & \mbox{ for }  j=1,\cdots,m\end{split}

In [1]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt

def distance(a,b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    return math.sqrt(dx*dx + dy*dy)

# Data
# Problem data


# ---------- K - MEDIAN ------------#

k = 2

subSections = [[0, 1.5],[1.5, 0.5],[0.75, 1.5],[1.2, 0.5],[1.7, 0.1],[0.87, 1.5],[1.42, 1.5]]
schools = [[1,0],[1,1],[1,2],
          [2,0],[2,1],[2,2]]

# import geopandas as gpd

# vagos = gpd.read_file('./data_gettin/vagos.geojson');
# centroids = vagos.centroid

# iteratorHandler = centroids.size
# centroidVector = []
# for centroid in centroids:
#     obj = [centroid.xy[0][0], centroid.xy[1][0]]
#     centroidVector.append(obj)

# subSections = centroidVector
# schools = centroidVector
# @charge: custo de construcao por escola (fixo em 500 mil euros)
# charge = 5000000

numSchools = len(schools)
numSubSections = len(subSections)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
d = {} # Distance Matrix
# @alpha: 0.29 de custo por Km  por (365 dias * 5 anos) 
alpha = 529.25

# creating binary variable for every school
for j in range(numSchools):
    x[j] = m.addVar(obj=0, vtype=GRB.BINARY, name="escola(%d)" % j)

# creating continuous variable for subsections to check suply fractions
for i in range(numSubSections):
    for j in range(numSchools):
        # Distances between Subsections and Schools
        d[(i,j)] = distance(subSections[i], schools[j])
        # Fractions of Subsection Suply
        y[(i,j)] = m.addVar(obj=d[i,j], vtype=GRB.BINARY, name="Demanda de Sub[%d] atendida pela escola[%d]" % (i,j))
        
m.update()

GurobiError: No Gurobi license found (user root, host fillipe-getin, hostid 5ba61100, cores 8)

In [ ]:
plt.scatter(*zip(*subSections))
plt.show()

In [ ]:
plt.scatter(*zip(*schools))
plt.show()

## Adding Constraints

In [ ]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)
for i in range(numSubSections):
    coef = [1 for j in range(numSchools)]
    var = [y[i,j] for j in range(numSchools)]
    m.addConstr(LinExpr(coef,var), "=", 1, name="Assign[%s]"%i)

# Constraint to set number of Schools equal to K
# m.addConstr(quicksum(x) == k)

coef = [1 for j in range(numSchools)]
var = [x[j] for j in range(numSchools)]
m.addConstr(LinExpr(coef,var), "=", rhs=k, name="k_median") 


# Fraction of suply must be lower or equal than the BINARY x[j] available 
# for i in range(numSubSections):
#     for j in range(numSchools):
#         m.addConstr(y[(i,j)] <= x[j])

for j in range(numSchools):
    for i in range(numSubSections):
        m.addConstr(y[i,j], "<", x[j], name="Strong[%s,%s]"%(i,j))

# THIS IS THE WRONG WAY TO ITERATE. DONT DO IT.
# for i in range(numSubSections):
#     for j in range(numSchools):
#         m.addConstr(quicksum(y[(i,j)]) <= x[j])


# Constraint to improve linear relaxation - RIGHT WAY!!!
# for j in range(numSchools):
#     m.addConstr(quicksum(y[(i,j)] for i in range(numSubSections)) <= x[j])

#### Objetive


In [ ]:
# Setting objective

m.setObjective( 
    quicksum(d[i,j]*y[i,j] for i in range(numSubSections) for j in range(numSchools)), GRB.MINIMIZE )

m.optimize()

In [ ]:
m

In [ ]:
print('Obj: %g' % m.objVal)

In [ ]:
for v in m.getVars():
    if(v.x != 0):
       print('%s   %g' % (v.varName, v.x))

In [ ]:
m = None

In [ ]:
disposeDefaultEnv()